# Tutorial 2: LipMIP
In this tutorial, we'll go over how to use LipMIP to exactly compute the Lipschitz constant of a ReLU network. LipMIP leverages Gurobi as a solver for mixed-integer programs, and you'll need to have gurobi installed (and gurobipy) for this section to work.

In [1]:
import sys 
sys.path.append('..')
import torch 
from pprint import pprint 

import utilities as utils 
from relu_nets import ReLUNet 
import neural_nets.data_loaders as data_loaders
import neural_nets.train as train 
from hyperbox import Hyperbox 
import interval_analysis as ia 
from lipMIP import LipMIP

## 1: Basic usage

In its simplest form, LipMIP frames evaluating the local Lipschitz constant of a scalar-valued ReLU network as a mixed-integer program. Crucially, there are two important components here: 
* Locality: We only support bounded hyperboxes as the domains in which we search over
* Scalar-valued: The ReLU network must output a scalar (we extend to multivariate networks later). For a typical neural net, $f$, which maps $\mathbb{R}^d\rightarrow \mathbb{R}^C$, we maintain a `c_vector`, $c$, and compute the Lipschitz constant of $\langle c, f\rangle$. For binary classification, $c$ is typically $[1,-1]$.

In [2]:
# As an example, we'll consider binary classification over R^2
DIMENSION = 2

# Define an input domain and c_vector 
simple_domain = Hyperbox.build_unit_hypercube(DIMENSION)
simple_c_vector = torch.Tensor([1.0, -1.0])

# Build a random network and a LipMIP instance 
network_simple = ReLUNet([2, 16, 16, 2])
simple_prob = LipMIP(network_simple, simple_domain, simple_c_vector, verbose=True, num_threads=2)

simple_result = simple_prob.compute_max_lipschitz()

Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 291 rows, 167 columns and 1181 nonzeros
Variable types: 139 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 1e+00]
  RHS range        [1e-06, 2e+00]
Presolve removed 145 rows and 76 columns
Presolve time: 0.01s
Presolved: 146 rows, 91 columns, 759 nonzeros
Variable types: 68 continuous, 23 integer (23 binary)

Root relaxation: objective 9.356497e-01, 117 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.93565    0   14          -    0.93565      -     -    0s
H    0     0                       0.2544534    0.93565   268%     -    0s
     0     0    0.91001    0   16    0.25445    0.91001   258%     -    0s
     0     0    0.79290    0   

In [3]:
# We can examine the result of this with the LipResult class 
simple_result

LipMIP Result: 
	Value 0.382
	Runtime 0.136

In [4]:
print(simple_result.as_dict().keys())  # A lot more information under the hood 
print('\n')

print(simple_result.squire.model) # like the gurobi.Model object we solved

dict_keys(['domain', 'value', 'sign_config', 'model', 'best_x', 'preacts', 'network', 'compute_time', 'c_vector', 'squire'])


<gurobi.Model MIP instance Unnamed: 291 constrs, 167 vars, Parameter changes: LogFile=gurobi.log, Threads=2, CSIdleTimeout=1800>


# 2: Stopping Early
There are two main techniques for early stopping which provides a certifiable **upper bound** to the lipschitz constant of a neural network. We describe how to include these options into LipMIP:

In [5]:
"""
Timeouts: If we only want to run LipMIP for a fixed clock time, this can be done by
          setting the `timeout` parameter (in seconds) upon initialization
Integrality_Gap: For maximization problems, MIP solves a sequence of relaxed problems which provide upper bounds
                 to the true optimum. Any feasible point provides a lower bound. Using this, Gurobi keeps track 
                 of an integrality gap (as an upper bound on percent error). i.e. if we want to be within 10% of 
                 the right answer, we can set `mip_gap` to 0.10
        
In either case of stopping early, the returned value is an UPPER BOUND to the true Lipschitz constant of a network.
"""
# More complicated network for which timeouts matter 
network_bigger = ReLUNet([10, 10, 10, 10, 10, 2])
bigger_domain = Hyperbox.build_unit_hypercube(10)
bigger_c_vector = torch.Tensor([1.0, -1.0])

timeout_prob = LipMIP(network_bigger, bigger_domain, bigger_c_vector, verbose=True, timeout=10.0)
timeout_result = timeout_prob.compute_max_lipschitz()

Changed value of parameter TimeLimit to 10.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 423 rows, 241 columns and 1679 nonzeros
Variable types: 195 continuous, 46 integer (46 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 2e+00]
  RHS range        [1e-06, 1e+00]
Presolve removed 96 rows and 42 columns
Presolve time: 0.00s
Presolved: 327 rows, 199 columns, 1364 nonzeros
Variable types: 155 continuous, 44 integer (44 binary)

Root relaxation: objective 3.682911e+00, 705 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.59732    0   36          -    3.59732      -     -    0s
H    0     0                       0.0215376    3.59732      -     -    0s
   

In [6]:
timeout_result

LipMIP Result: 
	Value 0.912
	Runtime 10.049

In [7]:
gap_prob = LipMIP(network_bigger, bigger_domain, bigger_c_vector, verbose=True, mip_gap=2)
gap_result = gap_prob.compute_max_lipschitz() # This should take less than a minute

Changed value of parameter MIPGap to 2.0
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 423 rows, 241 columns and 1679 nonzeros
Variable types: 195 continuous, 46 integer (46 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 2e+00]
  RHS range        [1e-06, 1e+00]
Presolve removed 96 rows and 42 columns
Presolve time: 0.01s
Presolved: 327 rows, 199 columns, 1364 nonzeros
Variable types: 155 continuous, 44 integer (44 binary)

Root relaxation: objective 3.682911e+00, 705 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.59732    0   36          -    3.59732      -     -    0s
H    0     0                       0.0215376    3.59732      -     -    0s
     0 

In [8]:
gap_result

LipMIP Result: 
	Value 0.141
	Runtime 202.191

# 3: Vector-valued Networks
As per section 7 and appendix E of our paper, we can extend LipMIP to evaluating the Lipschitz constant of vector-valued networks where the norms of interest are linear norms. That is, if $f:\mathbb{R}^n\rightarrow \mathbb{R}^m$ is a ReLU network, over some domain $\mathcal{X}$, we can compute 
$$ \max_{x,y\in\mathcal{X}} \frac{||f(x)-f(y)||_b}{||x-y||_a}$$
for some linear norms $||\cdot||_a, ||\cdot||_b$ over $\mathbb{R}^n, \mathbb{R}^m$ respectively. 

We describe the various norms our codebase can handle this below.

In [9]:
# First we handle the scalar-valued case. 
# The examples above have used abs(...) for the numerator and the L_inf norm for the denominator
# To consider the L_1 norm in the denominator, adjust the `primal_norm` parameter


# L-infinity norm (setting we use by default)
linf_problem = LipMIP(network_simple, simple_domain, simple_c_vector, primal_norm='linf', verbose=True) 
linf_problem.compute_max_lipschitz()

# L-1 norm
l1_problem = LipMIP(network_simple, simple_domain, simple_c_vector, primal_norm='l1', verbose=True) 
l1_problem.compute_max_lipschitz()

Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 291 rows, 167 columns and 1181 nonzeros
Variable types: 139 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [2e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 1e+00]
  RHS range        [1e-06, 2e+00]
Presolve removed 145 rows and 76 columns
Presolve time: 0.02s
Presolved: 146 rows, 91 columns, 759 nonzeros
Variable types: 68 continuous, 23 integer (23 binary)

Root relaxation: objective 9.356497e-01, 117 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.93565    0   14          -    0.93565      -     -    0s
H    0     0                       0.2544534    0.93565   268%     -    0s
H    0     0                       0.3816263    0.93565   145%     -    0s
     0     0    0.91001    0   

LipMIP Result: 
	Value 0.200
	Runtime 0.175

In [10]:
# Next we consider the vector valued cases:
# Case where both ||.||_a and ||.||_b are the infinity norms

inf_problem = LipMIP(network_simple, simple_domain, 'l1Ball1', verbose=True) # c_vector -> 'l1Ball1'
inf_problem.compute_max_lipschitz()

Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 294 rows, 171 columns and 1221 nonzeros
Variable types: 143 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 2e+00]
  RHS range        [1e-06, 5e+00]
Presolve removed 99 rows and 60 columns
Presolve time: 0.01s
Presolved: 195 rows, 111 columns, 956 nonzeros
Variable types: 88 continuous, 23 integer (23 binary)

Root relaxation: objective 3.505895e+00, 110 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.50589    0    2          -    3.50589      -     -    0s
H    0     0                       0.0718532    3.50589  4779%     -    0s
H    0     0                       0.1151976    3.50589  2943%     -    0s
     0     0    3.50589    0   

LipMIP Result: 
	Value 0.253
	Runtime 0.184

In [11]:
# Next we consider the case for CrossLipschitz robustness 
# We can encode these with c_vectors in the set 
# {'crossLipschitz', 'targetCrossLipschitz', 'trueCrossLipschitz', 'trueTargetCrossLipschitz'}
# where the parameters with the 'targeted' tag are for the targeted setting 
# and the 'true' tag is a slower but more precise cross-lipschitz norm 
cross_problem = LipMIP(network_simple, simple_domain, 'crossLipschitz', verbose=True) # c_vector -> 'l1Ball1'
cross_problem.compute_max_lipschitz()       

Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 294 rows, 171 columns and 1219 nonzeros
Variable types: 143 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [2e-04, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-06, 5e+00]
  RHS range        [1e-06, 1e+01]
Presolve removed 102 rows and 64 columns
Presolve time: 0.01s
Presolved: 192 rows, 107 columns, 946 nonzeros
Variable types: 84 continuous, 23 integer (23 binary)

Root relaxation: objective 3.505895e+00, 101 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.50589    0    2          -    3.50589      -     -    0s
H    0     0                       0.2418560    3.50589  1350%     -    0s
     0     0    3.50589    0    9    0.24186    3.50589  1350%     -    0s
H    0     0                  

LipMIP Result: 
	Value 0.382
	Runtime 0.183